In [1]:
# import sys
# import openai
# from langchain.llms import OpenAI
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
# from langchain.text_splitter import CharacterTextSplitter
# import re
# from concurrent.futures import ThreadPoolExecutor
# import logging
# from langchain.document_loaders import DirectoryLoader
import pandas as pd
import os
from langchain import OpenAI, VectorDBQA

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

risk_path=r"risk.xlsx"

### Hi DR, Remeber to input your OpenAI API Key in Above Cell !!!

In [2]:
risk_sor = pd.read_excel(risk_path)

In [3]:
# get the target text from original dataset to match
risk = pd.DataFrame(risk_sor["Abstract"])

risk.dropna(inplace=True, how="any")

In [4]:
def risk_template(words: str):
    return f"""help me to identify and extract risks in the '{words}'? please mark it out and specify the type . Also pick keywords to label what ENTIRE sentence talked about from given tags: [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance]. If you cannot judge the organizations or there is no matched tag to label the entire sentence, please return 'None'. Show risks, type of risks, keyword and tags appeared in sentence separately. Attention, you can only predict the output in above given criteria and no more than 15 words.No need to specific mark out on every sentence

    Answer example: "
    risks: Construction cost overrun, Ship collision and grounding \n

    risk_type : Financial, Technical \n

    keywords : Risk Management \n

    tags : Environment Science, Environment Studies, Finance" \n
    Answer no more than 20 words.
"""


def risk_template_backwords(words: str):
    return f"""help me to identify and extract risks in the '{words}'? please mark it out and specify the type . Also pick keywords to label what ENTIRE sentence talked about from given tags: [environment science, environment studies, engineering civil, engineering industrial, management, engineering manufacturing, constructing building, constructing technology, transportation, finance]. If you cannot judge the organizations or there is no matched tag to label the entire sentence, please return 'None'. Show type of risk follows its risks, keyword and tags appeared in sentence separately. Attention, you can only predict the category in above given criteria and no more than 20 words. No need to specific mark out on every sentence.
"""

In [5]:
def stk_keyword_split(res: list[str]):
    """res receive value from original feedback, right after OpenAI API return the value and split with \n"""
    feedback=[val.split(": ")[1] for val in res if val!='']
    if len(feedback)>3: feedback=[", ".join([feedback[0]+feedback[1]]), *feedback[2:]]
    elif len(feedback)<3: feedback=[*feedback, *[None]*(3-(len(feedback)))]
    return feedback

def risk_keyword_split(res: str):
    res=res.split("\n")
    feedback=[val.split(": ")[1] for val in res if val!='']
    if len(feedback)<4: feedback=[*feedback, *([None]*(4-len(feedback)))]
    return feedback

llm=OpenAI(temperature=0)

In [6]:
risk_df=pd.DataFrame([], columns=["risks description", "risk_type", "keywords", "sentence tags", "source"])

In [ ]:
risk_length, chunk_size=len(risk), 10
for times in range(2594, risk_length, chunk_size):
    long_phrase=risk.Abstract[times: times+chunk_size].to_list()
    words=risk_template(" ".join(long_phrase))
    try:
        response=llm(words)
        risk_df.loc[len(risk_df)]=[*risk_keyword_split(response), long_phrase]
    except Exception as e:
        print(f"the possible reason is {e}, be reminded, processed risk up to {times}")
        break